<a href="https://colab.research.google.com/github/ppw2004/colab_notebooks/blob/main/Evo_KPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 环境配置与依赖安装
如果在 Colab 或新环境中运行，请取消注释并执行安装命令。

In [ ]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install transformers accelerate peft datasets safetensors scipy scikit-learn pandas
# !pip install flash-attn --no-build-isolation  # 强烈推荐安装，显著加速 Evo

2. 导入库与参数配置

In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset, DatasetDict
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, average_precision_score
import torch.nn.functional as F

# =================== 核心配置 ===================
CONFIG = {
    # 路径设置
    "dataset_path": "./data/processed_datasets/dataset_meropenem_8k.parquet", # 先用 8k 调试，跑通后换 131k
    "model_id": "togethercomputer/evo-1-8k-base",  # 对应数据集版本 (8k 或 131k)
    "output_dir": "./results/evo_kpn_lora",

    # 训练参数
    "max_length": 8192,      # 必须与数据集切片长度一致 (8192 或 131072)
    "batch_size": 4,         # 根据显存调整 (A100可设大，3090需设小)
    "grad_accum": 8,         # 梯度累积，模拟大 Batch Size
    "learning_rate": 1e-4,   # LoRA 通常需要比全量微调更大的 LR [cite: 85]
    "epochs": 3,
    "lora_rank": 16,
    "lora_alpha": 32,

    # 损失函数参数
    "focal_gamma": 2.0,      # Focal Loss 关注难分样本的程度
    "beta": 0.9999           # Class Balanced Loss 的平滑因子
}

# 检查 GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Running on: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    # 启用 FlashAttention
    torch.backends.cuda.enable_flash_sdp(True)

3. 数据加载与防泄漏划分
关键步骤：为了防止“数据泄露”，我们必须按 genome_id 进行划分。绝不能简单随机划分，否则同一个细菌的切片会同时出现在训练集和测试集中，导致模型“背题”。

In [ ]:
def load_and_split_data(parquet_path):
    print(f"📖 Loading data from {parquet_path}...")
    df = pd.read_parquet(parquet_path)

    # 简单的 EDA
    n_pos = sum(df['label'] == 1)
    n_neg = sum(df['label'] == 0)
    print(f"📊 总样本数: {len(df)}")
    print(f"   耐药 (1): {n_pos} ({n_pos/len(df):.2%})")
    print(f"   敏感 (0): {n_neg} ({n_neg/len(df):.2%})")

    # 计算每个类别的样本数，供 Loss 函数使用
    samples_per_cls = [n_neg, n_pos]

    # --- Group Shuffle Split (防泄漏) ---
    splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, test_idx = next(splitter.split(df, groups=df['genome_id']))

    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    print(f"✅ 划分完成 (按 Genome ID):")
    print(f"   Train set: {len(train_df)} slices (Genomes: {train_df['genome_id'].nunique()})")
    print(f"   Test set:  {len(test_df)} slices (Genomes: {test_df['genome_id'].nunique()})")

    # 转换为 HuggingFace Dataset
    hf_train = Dataset.from_pandas(train_df.drop(columns=['__index_level_0__'], errors='ignore'))
    hf_test = Dataset.from_pandas(test_df.drop(columns=['__index_level_0__'], errors='ignore'))

    return DatasetDict({"train": hf_train, "test": hf_test}), samples_per_cls

dataset, samples_per_cls = load_and_split_data(CONFIG["dataset_path"])

4. 数据预处理 (Tokenization)
Evo 使用字节级 Tokenizer，我们需要将 DNA 字符串转换为 Input IDs。

In [ ]:
# 加载 Evo 的 Tokenizer (通常 Evo 使用自定义的，这里我们模拟 byte-level 处理)
# 注意：Evo 官方模型通常不需要由 transformers 提供的 AutoTokenizer，
# 但为了兼容 Trainer，我们定义一个简单的转换函数

def tokenize_function(examples):
    # Evo 的输入通常是 bytes 的 ASCII 编码
    # 我们需要将字符串 "ACGT..." 转换为对应的 tensor
    # Evo 词表设计：通常直接映射 bytes

    sequences = examples["sequence"]

    # 简单的 Byte-level 编码 (Evo 兼容)
    # 加上 batch 维度处理
    input_ids = []
    for seq in sequences:
        # 将字符串转为 ASCII 码列表 (0-255)
        ids = [ord(c) for c in seq]
        # 截断或 Padding 在 collator 中处理，或者这里做
        if len(ids) > CONFIG["max_length"]:
            ids = ids[:CONFIG["max_length"]]
        input_ids.append(ids)

    return {"input_ids": input_ids}

print("🔄 Tokenizing data...")
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["sequence", "genome_id", "chunk_id"], # 移除不需要的列
    num_proc=4
)
print("✅ Tokenization complete.")

5. 自定义模型架构 (EvoClassifier)
这是实验方案的核心：Backbone + Last Token Pooling + Classifier。

In [ ]:
class EvoClassifier(nn.Module):
    def __init__(self, model_name, num_labels=2, lora_config=None):
        super().__init__()
        self.num_labels = num_labels
        self.config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)

        print(f"🏗️ Loading Backbone: {model_name}")
        self.backbone = AutoModelForCausalLM.from_pretrained(
            model_name,
            config=self.config,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16, # 使用 BF16 防止溢出 [cite: 82]
            device_map="auto"
        )

        # 启用 Gradient Checkpointing 节省显存
        self.backbone.gradient_checkpointing_enable()

        # 应用 LoRA
        if lora_config:
            print("🔧 Applying LoRA...")
            self.backbone = get_peft_model(self.backbone, lora_config)
            self.backbone.print_trainable_parameters()

        # 自定义分类头 [cite: 41]
        self.classifier = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, num_labels)
        )

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Evo forward
        outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )

        # Last Token Pooling [cite: 40]
        # 提取最后一层的 hidden states
        last_hidden_state = outputs.hidden_states[-1]

        if attention_mask is not None:
            # 找到最后一个非 padding 的 token
            last_token_indices = attention_mask.sum(dim=1) - 1
            batch_size = last_hidden_state.shape[0]
            pooled_output = last_hidden_state[torch.arange(batch_size), last_token_indices]
        else:
            pooled_output = last_hidden_state[:, -1, :]

        # 预测
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            # Loss 计算将在 Trainer 中通过 CustomTrainer 处理
            # 这里仅返回 logits 供 Trainer 使用，或者直接在这里算标准 CrossEntropy
            # 为了兼容性，我们返回 logits
            pass

        return {"loss": loss, "logits": logits}

6. 损失函数：Class-Balanced Focal Loss
实现方案中提到的 CB-Focal Loss ，专门对付不平衡数据。

In [ ]:
class CBFocalLoss(nn.Module):
    def __init__(self, samples_per_cls, no_of_classes=2, gamma=2.0, beta=0.9999):
        super(CBFocalLoss, self).__init__()
        self.gamma = gamma
        self.beta = beta
        self.samples_per_cls = samples_per_cls
        self.no_of_classes = no_of_classes

        # 计算 Class-Balanced Weights
        effective_num = 1.0 - np.power(beta, samples_per_cls)
        weights = (1.0 - beta) / np.array(effective_num)
        weights = weights / np.sum(weights) * no_of_classes

        self.weights = torch.tensor(weights).float().to(device)

    def forward(self, logits, labels):
        # Focal Loss logic
        ce_loss = F.cross_entropy(logits, labels, reduction='none', weight=self.weights)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()

# 实例化 Loss
cb_focal_loss_fn = CBFocalLoss(samples_per_cls=samples_per_cls, gamma=CONFIG["focal_gamma"], beta=CONFIG["beta"])

7. 自定义 Trainer
覆盖 compute_loss 方法，注入我们的 CB-Focal Loss。

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # 使用自定义 Loss
        loss = cb_focal_loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

8. 评估指标 (Metrics)
重点关注 AUPRC 。

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Softmax 获取概率
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
    preds = np.argmax(logits, axis=-1)

    # 计算指标
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auroc = roc_auc_score(labels, probs)
    auprc = average_precision_score(labels, probs) # 核心指标

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auroc,
        'auprc': auprc  # 重点监控
    }

9. 开始训练

In [ ]:
# LoRA 配置
lora_config = LoraConfig(
    r=CONFIG["lora_rank"],
    lora_alpha=CONFIG["lora_alpha"],
    # StripedHyena 的特定层名称 [cite: 51]
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION # 或不指定，因为是自定义模型
)

# 初始化模型
model = EvoClassifier(CONFIG["model_id"], num_labels=2, lora_config=lora_config)
model.to(device)

# Data Collator (处理 Padding)
# 这里的 tokenizer 没有 pad_token，需要手动指定。Evo 通常用 N 或者特殊 token
# 简单的 Byte level padding 可以用 0 或其他
data_collator = DataCollatorWithPadding(tokenizer=AutoTokenizer.from_pretrained(CONFIG["model_id"], trust_remote_code=True))
# 注意：如果 AutoTokenizer 报错，需手动实现 collate_fn，将 list of input_ids pad 到相同长度

# 训练参数
training_args = TrainingArguments(
    output_dir=CONFIG["output_dir"],
    learning_rate=CONFIG["learning_rate"],
    per_device_train_batch_size=CONFIG["batch_size"],
    per_device_eval_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=CONFIG["grad_accum"],
    num_train_epochs=CONFIG["epochs"],
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=100, # 每100步验证一次
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="auprc", # 以 AUPRC 为准
    greater_is_better=True,
    bf16=True, # 必须使用 BF16 [cite: 82]
    dataloader_num_workers=4,
    report_to="none"
)

# 初始化 Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
    # data_collator=data_collator # 如果前面 tokenized 已经是定长，这里可以不用
)

print("🔥 Starting Training...")
trainer.train()

10. 模型保存与推理

In [ ]:
# 保存微调后的适配器
model.backbone.save_pretrained(os.path.join(CONFIG["output_dir"], "final_lora_adapter"))
torch.save(model.classifier.state_dict(), os.path.join(CONFIG["output_dir"], "classifier_head.pth"))
print("💾 Model saved.")

# 推理示例 (Inference)
def predict_genome(sequence):
    model.eval()
    # Tokenize
    input_ids = [ord(c) for c in sequence]
    input_tensor = torch.tensor([input_ids]).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.nn.functional.softmax(outputs['logits'], dim=-1)

    return probs[0][1].item() # 返回耐药概率

# 测试一条
print(f"Test Prediction (Resistant Prob): {predict_genome('ATCG'*100):.4f}")